In [1]:
import pickle
import os 
import cv2
import glob
import matplotlib.pyplot as plt

# select one of the multiple GPU cards on the server.
os.environ["CUDA_VISIBLE_DEVICES"]="0" # set GPU if multiple present




In [2]:
# the pkl directory contins the pkl files that hold a DeepLearning structure that defines the DL model
pklDir = input('Enter pkl path or use default\n') or '/media/amw/TOSHIBA EXT/alerding/models/pkl/'
# the model directory will receive the trained model
modelDir = input('Enter model output directory or default\n') or '/media/amw/TOSHIBA EXT/alerding/models/gpuModels/'
    
loadweights = None

# modelname is the name of hte pkl file without its pkl  extension.
# a pkl file holds a data structure or object. IN this case, it is expected to hold a DeepLearning object that was saved
# using File>Save DL Object... in mianalyzer. 
modelname = input("Enter modelname without extension or use default") or "unetDensnet201_2023-01-24_08-17-37"


numclasses = 4

# trainingdata is a directory that holds the annotated images. The labels must be in trainingdata/Segmentation_labels/
# and they must be npz files.
trainingdata = '/media/amw/TOSHIBA EXT/alerding/annotated/notCroppedLabels2/resized/'
validationdata = None

print("model: ", modelname)


model:  unetDensnet201_2023-01-24_08-17-37


In [ ]:
'''
This is left from the version that reads the dl object from a pkl file.
Now that the  dl structure can be defined in the code, it is not required.
'''

print('load settings')
#filehandler = open(modelname + '.pkl', 'rb')
filehandler = open(pklDir + modelname + '.pkl', 'rb')

dl = pickle.load(filehandler)

from pprint import pprint

pprint(vars(dl))


In [3]:
'''
Creates the dl object. Default is UNet, resnet152, 100 epochs,
scaleFactor is 0.5, and learning rate is 0.001.
All the other parmeters are the defaults of the dl object.
'''

# print(os.getcwd())

from dl.DeepLearning import *
def createDL( epochs=100, scaleFactor=0.5, learning_rate = 0.001):
    dl = DeepLearning()
    dl.Epochs = epochs
    dl.ImageScaleFactor = scaleFactor
#    dl.Mode = Segmentation()
    dl.learning_rate = learning_rate
    dl.Mode.architecture = 'UNet'
    dl.Mode.backbone = 'resnet152'
    return dl

In [ ]:
'''
create the dl object and verify that it is initialized. 
'''

dl = createDL( epochs=50, scaleFactor=0.5, learning_rate = 0.002)

# pod, stem, background, and unlabeled (for deleaved soybean plants)
numClasses =4

print('init model')
dl.initModel(numclasses)

dl.Model = dl.Mode.getModel(numClasses, 3)

print('load model')
if loadweights is not None:
    dl.Model.load_weights(loadweights)

print(dl.initialized)

In [ ]:
'''
Here we  do the training
'''
        
if dl.initialized:
    print('start training')
    dl.initData_StartTraining(trainingdata, validationdata)
    print('training finished')
else:
    print('could not initialize model')
            
print('saving weights')
dl.Model.save_weights(modelDir+modelname + '.h5')
print('saving training data')
dl.saveTrainingRecord(modelDir+modelname +'.csv')
#   print('saving model')
#   dl.Model.save(modelDir+'full')
    



In [4]:
%matplotlib inline

In [ ]:
# This cell does a prediction on a directory of jpg images (could be any type, but that is what I have).
# predictionFolder contains the images which ill be predicted, and a subdirectory with eh modelname will be created
# to hold the predicted segmentation.

predictionFolder = input('Enter folder for prediction or use default\n')  or '/home/dsi/aryeh/data/plants/Harvest8Orange5_7Oct17/'
outputPath = predictionFolder + modelname+'_ep'+str(dl.epochs)+'/'
try:
    os.mkdir(outputPath)
except FileExistsError:
    pass
files = glob.glob(predictionFolder + '*.jpg')
print(len(files))

# If we do not want to process the entire folder, we can limit the number of processed image to count. 
count = 0
for i in files:
    print(i)
    img = cv2.imread(i)
    prediction = dl.Mode.PredictImage(img)
    plt.figure(count)
    plt.imshow(prediction)
    plt.show()
    cv2.imwrite(outputPath+'segmented_'+os.path.basename(i).replace('jpg', 'png'), prediction)
    count += 1
    if count > 1:
       break
    



In [ ]:
# I was checking why repeated runs of the prediction produced different reauslts. One possibility is that
# I need to reset the DL object between runs. But I am not sure about this.
print(dl.Model)
dl.cleanMemory()